## **Extract Markov Process**

#### **Pre process log**

In [106]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.obj import EventLog, Trace, Event
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments


# --------------

log = xes_importer.apply("C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Assignments/2/log-10-percent-noise.xes.gz")
test_log = log[10:]
train_log = log[0:10]
def create_event_log(traces_list):
	log = EventLog()
	for trace_data in traces_list:
		trace = Trace()
		for event_data in trace_data[:-1]:
			event = Event(event_data)
			trace.append(event)
		log.append(trace)
	
	return log

train = create_event_log(train_log)
test = create_event_log(test_log)

# Extract Petri Net
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(train)
# pm4py.view_petri_net(net, initial_marking, final_marking)


# Perform alignments
aligned_traces = alignments.apply_log(train, net, initial_marking, final_marking)
al = [t[1] for t in aligned_traces[0]['alignment']]

# Build the perfect alignment
perfect_aligned_log = []
for aligned in aligned_traces:
	aligned_trace = []
	for move_log, move_model in aligned['alignment']:
		# keep synchronized moves
		if move_log not in [None, ">>"] and move_model not in [None, ">>"]:
			aligned_trace.append((move_log, move_model))
		
		# insert dummy for visible model moves (if present)
		elif move_log in [None, ">>"] and move_model not in [None, ">>", "tau"]:
			aligned_trace.append(f"{move_model}_DUMMY")
		
		# skip all other cases (trace-only moves or tau)
	
	perfect_aligned_log.append(aligned_trace)



parsing log, completed traces ::   0%|          | 0/100000 [00:00<?, ?it/s]

aligning log, completed variants ::   0%|          | 0/10 [00:00<?, ?it/s]

#### **Markov Process**

In [ ]:
import numpy as np
from copy import deepcopy
import math
from collections import defaultdict # mi permette di inizializzare un dizionario con valori di default (0 nel mio caso)
from pm4py.objects.petri_net import semantics

class Sample:
	def __init__(self, prefix, next_transition, trace_id, position):
		"""Represents a sample (x, y) where x is prefix and y is next transition"""
		self.prefix = prefix  # sequence of events before current position
		self.next_transition = next_transition  # transition at current position	
		self.trace_id = trace_id  # which trace this sample comes from
		self.position = position  # position in the alignment
	
class State:
	"""Represents a state in the Markov process"""
	def __init__(self, state_id, marking):
		self.state_id = state_id
		self.marking = marking
		self.samples = []  # List of Sample objects
		self.transitions = defaultdict(int)  # transition -> count
		self.total_samples = 0 # totale samples di ogni stato, mi serve come denominatore per calcolare le transition probabilities
	
	def add_sample(self, sample):
		self.samples.append(sample)
		self.transitions[sample.next_transition] += 1 # ogni volta che arriva in uno stato, ho la next_transition che arriva dal sample (che a sua volta arriva dal log aligned), sommo 1 ogni volta che compare questa next_transition
		self.total_samples += 1
	
	def get_transition_probabilities(self):
		"""Calculate transition probabilities from this state"""
		if self.total_samples == 0:
			return {}
		
		probabilities = {}
		for transition, count in self.transitions.items():
			probabilities[transition] = count / self.total_samples
		# print(f"State {self.state_id} transition probabilities: {probabilities}")
		return probabilities
		
	def calculate_entropy(self):
		"""Calculate entropy of this state"""
		probabilities = self.get_transition_probabilities()
		if not probabilities:
			return 0.0
		
		entropy = 0.0
		for prob in probabilities.values():
			if prob > 0:
				entropy -= prob * math.log2(prob)
		# print(f"State {self.state_id} entropy: {entropy}")
		return entropy


class MarkovProcess:
	def __init__(self, perfect_aligned_log, net, initial_marking, final_marking):
		self.perfect_aligned_log = perfect_aligned_log
		print("Perfect aligned log inside MarkovProcess:", self.perfect_aligned_log)
		self.net = net
		self.initial_state = None
		self.initial_marking = initial_marking
		self.final_marking = final_marking
		self.total_positions = self.calculate_tot_positions() # numero di tracce * cardinalità della traccia
		self.state_visit_counts = defaultdict(int) # per ogni stato, quante volte lo stato è stato visitato
		
		self.states = self.get_states()  # state_id -> State object
		# print("States: ", self.states)
		self.process_entropy = self.calculate_entropy()
		print("Process entropy: ", self.process_entropy)
		
	def get_states(self):
		print("Building states from aligned log...")
		
		state_counter = 0
		marking_to_state = {}  # marking -> state_id
		states = {}
		for trace_idx, alignment in enumerate(self.perfect_aligned_log):
			current_marking = self.initial_marking.copy()
			for pos, (trace_event, net_transition) in enumerate(alignment):
				if current_marking is None:
					# print(f"Skipping trace {trace_idx}, pos {pos}: marking is None")
					break   # oppure continue, a seconda di cosa vuoi fare
				
				# chiave dello stato dal marking
				marking_key = tuple(sorted((str(p), c) for p, c in current_marking.items())) # creo la chiave per ogni stato
				
				# Se lo stesso marking c'è già in un’altra traccia, non creo un nuovo stato, ma aggiungio il nuovo sample allo stato già esistente.
				if marking_key not in marking_to_state: # durante l'iterazione dentro la prima traccia entro in questo if e creo tutti gli stati
					state_id = f"S{state_counter}"
					state = State(state_id, current_marking.copy())
					states[state_id] = state # aggiungo lo stato al dizionario degli stati
					marking_to_state[marking_key] = state_id # salvo la corrispondenza marking -> state_id
					state_counter += 1 # semplicemente conta gli stati totali
				else:
					state_id = marking_to_state[marking_key]

				# crea sample e lo aggiungo al mio stato per capire che la traccia in questione (trace_idx) al passo pos si trova in questo stato
				prefix = []
				for i in range(pos):
					prev_event, prev_trans = alignment[i]
					if prev_event and prev_event not in {">>", "ε"}: # posso toglierla se considero il perfect log
						prefix.append(prev_event)
					else:
						print("SKIPPPP")

				if net_transition and net_transition != ">>": # posso toglierla se considero il perfect log
					sample = Sample(prefix, net_transition, trace_idx, pos)
					states[state_id].add_sample(sample)
					self.state_visit_counts[state_id] += 1
					# self.total_positions += 1 

					# aggiorna marking
					transition_obj = next((t for t in self.net.transitions if t.label == net_transition), None)
					if transition_obj:
						try:
							current_marking = semantics.execute(transition_obj, self.net, current_marking)
						except Exception:
							current_marking = None
				else:
					print("Ciao")

		return states
	
	def calculate_tot_positions(self):
		total_position = sum(len(log) for log in self.perfect_aligned_log)
		# print("Total positions calculated: ", total_position)
		return total_position
	
	def calculate_state_probability(self):
		# denominator = sum(len(trace) for trace in self.perfect_aligned_log) # positioni totali del log
		state_probabilities = {}
		for state, visit_count in self.state_visit_counts.items():
			state_probabilities[state] = visit_count / self.total_positions
			# print(f"State {state} has probability {round(state_probabilities[state], 3)}")
		return state_probabilities
	
	def calculate_entropy(self):
		process_entropy = 0.0
		state_probabilities = self.calculate_state_probability()
		# print("State probabilities:", state_probabilities)
		for state in self.states.values():
			state_entropy = state.calculate_entropy()
			state_probability = state_probabilities[state.state_id]
			process_entropy += state_probability * state_entropy
		return process_entropy
	

# Mantain only the alignement key, we do not take care of the other
aligned_log = []
for al in aligned_traces:
	aligned_log.append(al['alignment'])

markov_process = MarkovProcess(perfect_aligned_log, net, initial_marking, final_marking)

Perfect aligned log inside MarkovProcess: [[('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('X-Ray', 'X-Ray'), ('Check', 'Check'), ('Check', 'Check'), ('Visit', 'Visit'), ('Final Visit', 'Final Visit'), ('Check', 'Check')], [('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('Visit', 'Visit'), ('X-Ray', 'X-Ray'), ('Check', 'Check'), ('Final Visit', 'Final Visit')], [('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('X-Ray', 'X-Ray'), ('Visit', 'Visit'), ('Check', 'Check'), ('Check', 'Check'), ('Final Visit', 'Final Visit'), ('Prepare', 'Prepare')], [('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('X-Ray', 'X-Ray'), ('Visit', 'Visit'), ('Check', 'Check'), ('Check', 'Check'), ('Final Visit', 'Final Visit')], [('Triage', 'Triage'), ('Register', 'Register'), ('X-Ray', 'X-Ray'), ('Check', 'Check'), ('Visit', 'Visit'), ('Check', 'Check'), ('Check', 'Check'), ('Final Visit', 'Final Visit')], [('Triage', 'Triage'), 

## **Calculate information gain in process mining**

In [ ]:
import itertools

class EnrichedLog:
	def __init__(self, aligned_log, original_log, k):
		"""
		Crea un log arricchito a partire dal log originale e da quello allineato,
		con timestamp, label originale e label binaria di lunghezza k.

		Parameters
		----------
		aligned_log : list
			Output di pm4py.conformance.alignments.apply_log
		original_log : list
			Log originale (pm4py log)
		k : int
			Dimensione del codice binario (0,1)^k
		"""
		self.k = k
		self.label_to_code = {}  # mapping globale label -> codice
		self.code_index = 0
		self.all_codes = list(itertools.product([0, 1], repeat=k))

		# Costruisco il log arricchito direttamente
		self.enriched_log = self._build_enriched_log(aligned_log, original_log)

	def _build_enriched_log(self, aligned_log, original_log):
		enriched_log = []

		for trace_idx, alignment_info in enumerate(aligned_log):
			alignment = alignment_info
			original_trace = original_log[trace_idx]

			enriched_trace = []
			log_event_idx = 0

			for aligned_event, model_transition in alignment:
				if aligned_event != ">>":  # skip delle mosse nel modello
					log_event = original_trace[log_event_idx]

					# assegna codice binario coerente alla label
					lbl = log_event["concept:name"]
					if lbl not in self.label_to_code:
						if self.code_index >= len(self.all_codes):
							raise ValueError(
								f"Non ci sono abbastanza codici binari per {lbl}, aumenta k"
							)
						self.label_to_code[lbl] = self.all_codes[self.code_index]
						self.code_index += 1

					enriched_trace.append({
						"original_label": lbl,
						"binary_label": self.label_to_code[lbl],
						"timestamp": log_event["time:timestamp"].timestamp(),
						"model_transition": model_transition
					})
					log_event_idx += 1

			enriched_log.append(enriched_trace)

		return enriched_log

k = 3 # lunghezza codifica binaria 
enriched_log_obj = EnrichedLog(aligned_log, log, k) 
enriched_log_obj.enriched_log[0]

In [ ]:
class InformationGain():
	def __init__(self, markov_process, psi, threshold, enriched_log):
		self.psi = psi # is an array with len = k, [0,0,0] or [1,0,1] or [1] or [,1,0] ecc.
		self.threshold = threshold
		self.markov_process = markov_process
		self.enriched_log = enriched_log
		self.z_true ,self.z_false = self.calculate_datasets()
		self.z = self.union_of_dicts()
		self.entropy_y_true = self.calculate_entropy_y(self.z_true)
		self.entropy_y_false = self.calculate_entropy_y(self.z_false)
		
		self.entropy_trace_test = self.calculate_entropy_trace_test()
		self.information_gain = self.calculate_information_gain()
	
	def calculate_datasets(self):
		z_true = defaultdict(list)
		z_false = defaultdict(list)
		for key, state in self.markov_process.states.items():
			for sample in state.samples:
				# if len(sample.prefix) > 0:
				if (self.evaluate_trace(sample)):
					z_true[state.state_id].append(sample)
					# z_false[state.state_id].append(None)
					# print("Aggiunto sample a z_true")
				else:
					# print("Aggiunto sample a z_false")
					z_false[state.state_id].append(sample)
					# z_true[state.state_id].append(None)
		return z_true, z_false
	
	def union_of_dicts(self):
		merged = {}
		for d in [self.z_true, self.z_false]:
			for k, v in d.items():
				merged.setdefault(k, []).append(v)
		return merged
	
	def evaluate_trace(self, sample):
		# print(prefix)
		prefix = sample.prefix
		next_transition = sample.next_transition
		pos = sample.position
		trace_idx = sample.trace_id
		# print(pos)
		
		# get timestamps and labels, labels are array of array because each label is an array like [0,1,1]
		timestamps, labels = self.get_timestamps_labels_from_enriched_log(prefix, next_transition, pos, trace_idx)
		last_timestamp = timestamps[-1]
		for timestamp, label in zip(timestamps, labels):
			# check timestamp threshold
			# print("Last timestamp: ", last_timestamp)
			# print("Timestamp: ", timestamp)
			if last_timestamp - timestamp <= self.threshold:
				# check trace test hold
				# print("Difference between timestamps less than threshold...")
				if (self.check_satisfiability(label)):
					return True
			# else: 
				# do nothing
		return False
	
	def get_timestamps_labels_from_enriched_log(self, prefix, next_transition, pos, trace_idx):
		trace_in_log = self.enriched_log.enriched_log[trace_idx]
		timestamps = []
		labels = []
		
		last_event_of_sample = trace_in_log[pos]
		
		for i in range(pos + 1):
			timestamp = trace_in_log[i]['timestamp']
			label = trace_in_log[i]['binary_label']
			timestamps.append(timestamp)
			labels.append(label)
		
		return timestamps, labels
	
	def check_satisfiability(self, label):
		# print("Label: ", label)
		for index, pos in enumerate(self.psi):
			# print("Pos: ", pos)
			if (pos != -1):
				if pos != label[index]:
					return False
		return True
	
	def calculate_entropy_y(self, z):
		entropy_y = {}
		for state_id, samples in z.items():
			entropy_y[state_id] = self.calculate_entropy_for_samples(samples) # calcolo entropia in base ai samples
		return entropy_y

	def calculate_entropy_for_samples(self, samples):
		"""Calcola l'entropia su un sottoinsieme di sample (ZQ^⊤ o ZQ^⊥)."""
		if not samples:
			return 0.0

		# Conta le transizioni future (y)
		counts = defaultdict(int)
		for sample in samples:
			counts[sample.next_transition] += 1

		total = len(samples)
		entropy = 0.0
		for count in counts.values():
			p = count / total
			entropy -= p * math.log2(p)

		return entropy

	def calculate_entropy_trace_test(self):
		entropy_trace_test = 0.0
		state_probabilities = self.markov_process.calculate_state_probability()
		for state in self.markov_process.states:
			if (len(self.z_false[state]) != 0 and len(self.z_true[state]) != 0):
				entropy_trace_test += state_probabilities[state] * ((len(self.z_true[state])/len(self.z[state])) * self.entropy_y_true[state] + (len(self.z_false[state])/len(self.z[state])) * self.entropy_y_false[state])
		return entropy_trace_test
		
	def calculate_information_gain(self):
		return self.markov_process.process_entropy - self.entropy_trace_test

psi = [0,1,1] # -1 for the position not interested
threshold = 0
k = 3

markov_process_trace_test = MarkovProcess(perfect_aligned_log, net, initial_marking, final_marking)
enriched_log_obj = EnrichedLog(aligned_log, log, k)

print("\nTrace test...")
trace_test = InformationGain(markov_process=markov_process_trace_test, psi=psi, threshold=threshold, enriched_log=enriched_log_obj)
print("Entropy of true partition: ", trace_test.entropy_y_true)
print("Entropy of false partition: ", trace_test.entropy_y_false)
print("Entrpy of trace test: ", trace_test.entropy_trace_test)
print("Entropy of Markov process:", trace_test.markov_process.process_entropy) 
print("Information gain: ", trace_test.information_gain)

Perfect aligned log inside MarkovProcess: [[('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('X-Ray', 'X-Ray'), ('Check', 'Check'), ('Check', 'Check'), ('Visit', 'Visit'), ('Final Visit', 'Final Visit'), ('Check', 'Check')], [('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('Visit', 'Visit'), ('X-Ray', 'X-Ray'), ('Check', 'Check'), ('Final Visit', 'Final Visit')], [('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('X-Ray', 'X-Ray'), ('Visit', 'Visit'), ('Check', 'Check'), ('Check', 'Check'), ('Final Visit', 'Final Visit'), ('Prepare', 'Prepare')], [('Triage', 'Triage'), ('Register', 'Register'), ('Check', 'Check'), ('X-Ray', 'X-Ray'), ('Visit', 'Visit'), ('Check', 'Check'), ('Check', 'Check'), ('Final Visit', 'Final Visit')], [('Triage', 'Triage'), ('Register', 'Register'), ('X-Ray', 'X-Ray'), ('Check', 'Check'), ('Visit', 'Visit'), ('Check', 'Check'), ('Check', 'Check'), ('Final Visit', 'Final Visit')], [('Triage', 'Triage'), 